In [2]:
import correctionlib
import hist
import awkward as ak
import numpy as np

from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.lookup_tools.correctionlib_wrapper import correctionlib_wrapper
from coffea.lookup_tools.dense_lookup import dense_lookup

In [3]:
events = NanoEventsFactory.from_root(
    "root://cmsxrootd-site.fnal.gov//store/mc/RunIISummer20UL18NanoAODv9/TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/120000/0520A050-AF68-EF43-AA5B-5AA77C74ED73.root",
    entry_stop=100_000,
    schemaclass=NanoAODSchema,
).events()

In [4]:
phasespace_cuts = (
    (abs(events.Jet.eta) < 2.5)
    & (events.Jet.pt > 40.)
)
jets = ak.flatten(events.Jet[phasespace_cuts])

efficiencyinfo = (
    hist.Hist.new
    .Reg(20, 40, 300, name="pt")
    .Reg(4, 0, 2.5, name="abseta")
    .IntCat([0, 4, 5], name="flavor")
    .Bool(name="passWP")
    .Double()
    .fill(
        pt=jets.pt,
        abseta=abs(jets.eta),
        flavor=jets.hadronFlavour,
        passWP=jets.btagDeepFlavB > 0.2783, # UL 2018 medium WP
    )
)
efficiencyinfo

Hist(
  Regular(20, 40, 300, name='pt', label='pt'),
  Regular(4, 0, 2.5, name='abseta', label='abseta'),
  IntCategory([0, 4, 5], name='flavor', label='flavor'),
  Boolean(name='passWP', label='passWP'),
  storage=Double()) # Sum: 331859.0 (334836.0 with flow)

In [5]:
eff = efficiencyinfo[{"passWP": True}] / efficiencyinfo[{"passWP": sum}]
# note this seems to turn 0,4,5 into 0,1,2
efflookup = dense_lookup(eff.values(), [ax.edges for ax in eff.axes])
efflookup

3 dimensional histogram with axes:
	1: [ 40.  53.  66.  79.  92. 105. 118. 131. 144. 157. 170. 183. 196. 209.
 222. 235. 248. 261. 274. 287. 300.]
	2: [0.    0.625 1.25  1.875 2.5  ]
	3: [0. 1. 2. 3.]

In [7]:
# Efficiency at 42 GeV, |eta|=0.2, for light, c, and b quark respectively
efflookup(np.array([42,60]), 0.2, 2)

array([0.83132001, 0.84244562])

In [6]:
cset = correctionlib.CorrectionSet.from_file("/cvmfs/cms.cern.ch/rsync/cms-nanoAOD/jsonpog-integration/POG/BTV/2018_UL/btagging.json.gz")
print([c for c in cset])
# more docs at https://cms-nanoaod-integration.web.cern.ch/commonJSONSFs/BTV_btagging_Run2_UL/BTV_btagging_2018_UL.html

['deepCSV_comb', 'deepCSV_incl', 'deepCSV_mujets', 'deepCSV_shape', 'deepJet_comb', 'deepJet_incl', 'deepJet_mujets', 'deepJet_shape']


In [7]:
def lighttagSF(j, syst="central"):
    # until correctionlib handles jagged data natively we have to flatten and unflatten
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepJet_incl"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)


def btagSF(j, syst="central"):
    # until correctionlib handles jagged data natively we have to flatten and unflatten
    j, nj = ak.flatten(j), ak.num(j)
    sf = cset["deepJet_comb"].evaluate(syst, "M", np.array(j.hadronFlavour), np.array(abs(j.eta)), np.array(j.pt))
    return ak.unflatten(sf, nj)

In [8]:
lightJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour == 0)]
bcJets = events.Jet[phasespace_cuts & (events.Jet.hadronFlavour > 0)]

lightEff = efflookup(lightJets.pt, abs(lightJets.eta), lightJets.hadronFlavour)
bcEff = efflookup(bcJets.pt, abs(bcJets.eta), bcJets.hadronFlavour)

In [9]:
# BTagSFMethod "1a"

def combine(eff, sf, passbtag):
    # tagged SF = SF*eff / eff = SF
    tagged_sf = ak.prod(sf[passbtag], axis=-1)
    # untagged SF = (1 - SF*eff) / (1 - eff)
    untagged_sf = ak.prod(((1 - sf*eff) / (1 - eff))[~passbtag], axis=-1)
    return tagged_sf * untagged_sf

lightweight = combine(
    lightEff,
    lighttagSF(lightJets),
    lightJets.btagDeepB > 0.2783,
)
bcweight = combine(
    bcEff,
    btagSF(bcJets),
    bcJets.btagDeepB > 0.2783,
)
eventweight = lightweight * bcweight
eventweight

<Array [0.986, 1.67, 1.15, ... 1.13, 0.982] type='100000 * float64'>

In [10]:
eventweight_lightflavUp = combine(
    lightEff,
    lighttagSF(lightJets, "up"),
    lightJets.btagDeepB > 0.2783,
) * bcweight

eventweight_lightflavUp / eventweight

<Array [0.992, 1.15, 0.997, ... 0.99, 1, 0.99] type='100000 * float64'>